In [1]:
import pandas as pd
try:
    from StringIO import StringIO
except ImportError:
    from io import StringIO
import numpy as np

In [ ]:
mylist = []
for chunk in pd.read_csv(r'nflscrapR-data/play_by_play_data/regular_season/reg_pbp_2018.csv', sep=',', chunksize=20000,low_memory=False):
    mylist.append(chunk)
p18 = pd.concat(mylist, axis = 0)
del mylist

In [ ]:
#tackles
#tacklescoring - https://imgur.com/Ig0eCsh
#tacklepositions - LB, S, QB, RB, WR, TE, PK, PN
#groupby players and their tackles, create column to determine points from tackles, join with roster data, filter by eligible positions, 
tacklepts18one = pd.DataFrame({'tackles' : p18.groupby( [ "assist_tackle_1_player_id"] ).size()}).reset_index()
tacklepts18two = pd.DataFrame({'tackles' : p18.groupby( [ "assist_tackle_2_player_id"] ).size()}).reset_index()

In [ ]:
#gotta figure out how to merge them, something about the column names not being indexed properly
tacklepts18 = pd.merge(tacklepts18one, tacklepts18two, left_on = 'assist_tackle_1_player_id', right_on = 'assist_tackle_2_player_id', how='inner')

In [5]:
#Add both groups of tackle assists together, multiply by 0.5 for fantasy point total
tacklepts18['tackles_xy'] = (tacklepts18['tackles_x'] + tacklepts18['tackles_y']) * 0.5
tacklepts18.head()

,assist_tackle_1_player_id,tackles_x,assist_tackle_2_player_id,tackles_y,tackles_xy
0,00-0021140,7,00-0021140,4,5.5
1,00-0022161,5,00-0022161,6,5.5
2,00-0022247,14,00-0022247,19,16.5
3,00-0023173,2,00-0023173,1,1.5
4,00-0023259,13,00-0023259,13,13.0


In [6]:
print(len(tacklepts18))

745


In [8]:
rosters18 = pd.read_csv(r'rosters_2018.csv', sep = ',', low_memory = False)

In [9]:
print(len(tacklepts18))

745


In [15]:
rosters18 = rosters18.merge(tacklepts18[['tackles_xy','assist_tackle_1_player_id']],left_on='GSIS_ID',right_on='assist_tackle_1_player_id')
rosters18.head()

KeyError: "['qb_hit_1_player_id'] not in index"

In [12]:
rosters18.drop(['assist_tackle_1_player_id'], axis=1,inplace=True)

KeyError: "['assist_tackle_1_player_id'] not found in axis"

In [17]:
rosters18.head()
print(len(rosters18))

55


In [14]:
qbhit18one = pd.DataFrame({'qb_hit' : p18.groupby( [ "qb_hit_1_player_id"] ).size()}).reset_index()
qbhit18two = pd.DataFrame({'qb_hit' : p18.groupby( [ "qb_hit_2_player_id"] ).size()}).reset_index()

In [15]:
qbhitpts18 = pd.merge(qbhit18one, qbhit18two, left_on = 'qb_hit_1_player_id', right_on = 'qb_hit_2_player_id', how='inner')

In [16]:
qbhitpts18['qb_hit_xy'] = qbhitpts18['qb_hit_x'] + qbhitpts18['qb_hit_y']
qbhitpts18.head()

,qb_hit_1_player_id,qb_hit_x,qb_hit_2_player_id,qb_hit_y,qb_hit_xy
0,00-0022161,13,00-0022161,2,15
1,00-0023368,16,00-0023368,1,17
2,00-0024338,2,00-0024338,1,3
3,00-0025828,6,00-0025828,1,7
4,00-0026142,19,00-0026142,1,20


In [17]:
rosters18.head()
print(len(rosters18))

55


In [18]:
rosters18 = rosters18.merge(qbhitpts18[['qb_hit_xy','qb_hit_1_player_id']],left_on='GSIS_ID',right_on='qb_hit_1_player_id')
rosters18.head()

,player_id,pos,name,short_name,tackles_xy,qb_hit_xy,qb_hit_1_player_id
0,00-0027010,S,Patrick Chung,P.Chung,17.0,2,00-0027010
1,00-0030573,T,Sheldon Richardson,S.Richardson,12.0,16,00-0030573
2,00-0027949,DE,J.J. Watt,J.Watt,9.0,25,00-0027949
3,00-0029249,DE,Melvin Ingram,M.Ingram,8.5,16,00-0029249


In [19]:
print(len(rosters18))

4
